In [144]:
# -*- coding: utf-8 -*-
import numpy as np
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import font, filedialog, messagebox
from tkinter.filedialog import asksaveasfile
from tkinter.messagebox import *
import json
from metricsyso import *
import pandas as pd
import time
from tqdm import tqdm

class MyApp():
    def __init__(self,windows):
        self.windows = windows
        windows.title("My App")
        self.conf_df = None
        self.df_m =None 
        self.df_w =None
        
        # configure grid layout 
        windows.rowconfigure(0, weight=1)
        windows.columnconfigure(0,weight=1)
        
        for i in range(12):
            for j in range(4):
                 windows.rowconfigure(i, weight=1)
                 windows.columnconfigure(j, weight=1)
             
        # frame for getting ground truth file
        self.gtruth_file_entry = Entry(windows, width=50)
        self.gtruth_file_entry.grid(row=1,column=1, columnspan=2, pady=15)
        self.gtruth_file_btn = tk.Button(windows, text="Load Ground Truth File", command=lambda: self.gtruth_file_entry.insert(tk.END, self.browse_file("Select Ground Truth File")))
        self.gtruth_file_btn.grid(row=0,column=1, columnspan=2, pady=15)
        
        # fram e for getting prediction file
        self.pred_file_entry = Entry(windows, width=50)
        self.pred_file_entry.grid(row=3, column=1, columnspan=2, pady=5)
        self.pred_file_btn = tk.Button(windows, text="Load Prediction File", command=lambda: self.pred_file_entry.insert(tk.END, self.browse_file("Select Prediction File")))
        self.pred_file_btn.grid(row=2, column=1, columnspan=2, pady=5)
        
        # frame for geting entry for json file
        self.json_file_entry = Entry(windows, width=50)
        self.json_file_entry.grid(row=5, column=1, columnspan=2, pady=5)
        self.json_file_btn = tk.Button(windows, text="Load JSON File", command=lambda: self.json_file_entry.insert(tk.END, self.browse_file("Select JSON File")))
        self.json_file_btn.grid(row=4, column=1, columnspan=2, pady=5) 
        # Displaying buttons for metrics
        # dispalyfram
        """self.disframe=Frame(windows, borderwidth=2,bg='white', bd=3, cursor='hand2', height=50, 
                              highlightcolor='blue', highlightthickness=2, highlightbackground='black', 
                              relief=tk.GROOVE)  
        self.disframe.grid(row=3, column=0)"""
        #self.button1 = Button(windows,text="Display confusion matrix", command =self.display_confmatrix).place(x=60, y=200,height = 40, width = 180)
        self.button1 = Button(windows, text="Display confusion matrix", command =self.display_confmatrix)
        self.button2 = Button(windows, text="Display metrics", command =self.display_df_metrics)
        self.button3 = Button(windows, text="Display weighted metrics", command =self.display_weightetd_metrics)
        self.buttonclose = Button (windows, text ="Exit", width=3, command = self.close_window)
        self.save_button = tk.Button(windows, text="Save to File", command=self.save_to_file)
        
        #pack in grid
        self.button1.grid(row=2,column=0)
        self.button2.grid(row=3,column=0)
        self.button3.grid(row=4,column=0)
        self.save_button.grid(row=9, column=1, columnspan=2, pady=15)
        self.buttonclose.grid(row=9, column=3, columnspan=2, pady=10) 
        
        # Create a frame for displaying
        self.frame = tk.Frame(windows, borderwidth=2,bg='white', bd=3, cursor='hand2', height=50, 
                              highlightcolor='blue', highlightthickness=2, highlightbackground='black', 
                              relief=tk.RAISED)
        self.frame.grid_propagate(True)
        self.frame.grid(row=7,column=1, columnspan=2, pady=5, sticky='NS')
        
        self.default_dict = default_dict = {"1":"default","2":"ground","5": "vegetation", 
                "6":"building","7": "wall", 
                "196": "structure","211":"cable"
                }
        #Label(self.frame, bg="white").pack(padx=10, pady=10)
        #self.button = Button(windows,text="Display confsusion matrix", command =self.compute_metrics).place(x=60, y=45,height = 50, width = 180)
        
        # Create buttons to compute metrics
        self.compute_button = tk.Button(windows, text="Compute Metrics",command=lambda: self.compute_metrics(self.gtruth_file_entry.get(),self.pred_file_entry.get(),self.json_file_entry.get()))
        self.compute_button.grid(row=0, column=0)
        # place a progressbar
        self.pb = ttk.Progressbar(windows, orient='horizontal',    mode='indeterminate',    length=280)
        self.pb.grid(row=1, column=0)
        

    
        # Create a treeview in the frame for dataframe pandas
        
        self.df_tree = ttk.Treeview(self.frame)
        self.df_tree.grid(row=0,column=0)
        
        # ScrollBar for the frame an d treeview
        
        # horizontal scrollbar
        self.horscrolbar = Scrollbar(self.frame, orient='horizontal', command= self.df_tree.xview) 
        self.horscrolbar.grid(row=1, column=0)
        
        # vertical scrollbar
        self.verscrolbar =Scrollbar(self.frame, orient='vertical',command= self.df_tree.yview)
        self.verscrolbar.grid(row=0, column=1)
        #self.df_tree.tag_configure('ttk', background='yellow'
        self.df_tree.configure(xscrollcommand= self.horscrolbar.set,yscrollcommand= self.verscrolbar.set)
        
        
        
        #self.status_label = tk.Label(windows, text="", padx=20, pady=10)
        #self.status_label.pack()
        
        
        
        #self.text_widget = tk.Text(windows, wrap=tk.WORD)
        #self.text_widget.pack(padx=20, pady=20, fill="both", expand=True)
        
        """self.hscrolbar = Scrollbar(self.frame, orient='horizontal')
        self.hscrolbar.pack(side=BOTTOM, fill = X)
        self.vscrolbar = Scrollbar(self.frame, orient='vertical')
        self.vscrolbar.pack(side = RIGHT, fill = Y)"""
        
        
        


    def browse_file(self,title):
        filepath = filedialog.askopenfilename(title=title, filetypes=(("Laz or Las files", "*.laz *.las"),
                                                                    ("All files", "*.*") ,
                                                                    ("Json files","*.json")))
        return filepath


    def tree_display(self,df):
            print(df)
            self.df_tree.delete(*self.df_tree.get_children())
            
            #df_list=list(df.columns.values)
            
            df_rset=df.to_numpy().tolist()
            #df['index'] = df.index
            
            #self.df_tree=ttk.Treeview(self.frame,columns=df_list)
            #self.df_tree.pack()                                      
            """for i in df_list:
                self.df_tree.column(i,width=100,anchor='c')
                self.df_tree.heading(i,text=i)"""
                
            columns= list(df.columns[:])
            #columns1 =np.concatenate(df.index.name,columns)
            self.df_tree["columns"] = columns
            self.df_tree ["show"] = "headings"
            for column in columns :
                self.df_tree.heading (column , text= column)
                self.df_tree.column(column, width= 100, anchor = 'c')
                
            for dt in df_rset:
                v=[r for r in dt]
                self.df_tree.insert('','end', values=v)
    

    def compute_metrics(self,gtruth_file,pred_file,json_file):
        
        #gtruth_file =self.gtruth_file_entry.get()
        #pred_file = self.pred_file_entry.get()
        #json_file = self.json_file_entry.get()
        
        
        # Reset pb bar
        self.pb['value'] = 0
        self.windows.update_idletasks()
        
        # step 1: Load Json file
        op = JsonFile(json_file)
        conv_dict = op.convdict()
        conv_class = op.convclass()
        truth_class = op.conv_to_Truclass(conv_dict, conv_class)
        self.pb['value'] = 20
        self.windows.update_idletasks() 

        # Step 1 : read gtruth file and  pred file
        gtruth = read_file_class(gtruth_file)
        prediction = read_file_class(pred_file)
        count_pred = num_points(prediction)
        count_gtruth = num_points(gtruth)
        self.pb['value'] = 40
        self.windows.update_idletasks()
        
        # step 3 : convert gtruth classes
        convertor = truth_class
        gtruth_conv = [convertor.get(str(k)) if convertor.get(str(k)) is not None else k for k in count_gtruth]
        self.pb['value'] = 60
        self.windows.update_idletasks()

        # step 4 : conpute confusion matrix
        input_pred = np.array(count_pred)
        input_gtruth = np.array(gtruth_conv)
        unique_classes =  np.unique(np.concatenate((input_gtruth, input_pred )))
        self.clas_name = [str(name) for name in tqdm(unique_classes)]
        self.conf_df= compute_confusion_matrix(input_gtruth, input_pred)
        self.conf_df.rename(columns = self.default_dict, index = self.default_dict, inplace = True)
        self.pb['value'] = 80
        self.windows.update_idletasks()
        
        # Step 5 compute metrics
        m = Metrics(input_gtruth, input_pred)
        metrics_fr = m.compute_metrics()
        self.df_m =m.metrics_frame(metrics_fr)
        self.df_m.rename(columns = self.default_dict, index = self.default_dict, inplace = True)
        self.df_w = m.weighted_metrics()
        self.pb['value'] = 100
        self.windows.update_idletasks()

    

    # displayinf data frame witrh a treeView
    
    def display_confmatrix(self):
        self.tree_display(self.conf_df)

    def display_df_metrics(self):
        self.tree_display(self.df_m)

    def display_weightetd_metrics(self):
        self.tree_display(self.df_w)
                    

    def save_to_file(self):
        files = [('All Files', '*.*'),  
             ('Text Document', '.txt'),
             ('Excel files', '.xlsx')] 
        self.file_path = asksaveasfile(filetypes = files, defaultextension ='.xlsx')
        self.save_frame()
        
    def save_frame(self):
        if self.display_confmatrix:
            if self.file_path:
                try:
                    with pd.ExcelWriter(self.file_path.name) as writer:
                        if hasattr(self, 'conf_df') and isinstance(self.conf_df,pd.DataFrame):
                            self.conf_df.to_excel(writer,sheet_name= "Confusion_Matrix", index=True)
                            messagebox.showinfo("Succes", f"Data saved to {self.file_path.name}")
                except Exception as e:
                    messagebox.showerror("Error", f"Failed to save data:{str(e)}")
        
        elif self.display_df_metrics:
            if self.file_path:
                try:
                    with pd.ExcelWriter(self.file_path.name) as writer:
                        if hasattr(self, 'df_m') and isinstance(self.df_m,pd.DataFrame):
                            self.df_m.to_excel(writer, sheet_name= "DataFrame_Metrics", index=True)
                            messagebox.showinfo("Succes", f"Data saved to {self.file_path.name}")
                except Exception as e:
                    messagebox.showerror("Error", f"Failed to save data:{str(e)}")
        
        elif self.display_weightetd_metrics:
            if self.file_path:
                try:
                    with pd.ExcelWriter(self.file_path) as writer:
                        if hasattr(self, 'df_w') and isinstance(self.df_w,pd.DataFrame):
                            self.df_w.to_excel(writer,sheet_name= "Weighted_Metrics", index=True)
                            messagebox.showinfo("Succes", f"Data saved to {self.file_path.name}")
                except Exception as e:
                    messagebox.showerror("Error", f"Failed to save data:{str(e)}")
    
    
    def close_window(self):
        self.windows.destroy()
        
        
windows =Tk()
my_gui = MyApp(windows)
windows.geometry('900x600')
#askyesnocancel("askyesnocancel", "Yes or No or Cancel")
windows.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\ysouley\AppData\Local\Temp\ipykernel_50256\337559470.py", line 84, in <lambda>
    self.compute_button = tk.Button(windows, text="Compute Metrics",command=lambda: self.compute_metrics(self.gtruth_file_entry.get(),self.pred_file_entry.get(),self.json_file_entry.get()))
                                                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ysouley\AppData\Local\Temp\ipykernel_50256\337559470.py", line 176, in compute_metrics
    conv_dict = op.convdict()
                ^^^^^^^^^^^^^
  File "m:\#Utilisateurs\YSO\YSO_scripts\metricsyso.py", line 298, in convdict
    f = json.load(file)
        ^^^^^^^^^^^^^^^

len ground_truth 11292247
len pred 11292247
min length11292247


100%|██████████| 11292247/11292247 [00:39<00:00, 283907.54it/s]
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisio

           default   building     50        60     64       67         68  \
Classes                                                                     
default        0.0        0.0  780.0       0.0  157.0      0.0        0.0   
building       0.0  6279367.0    0.0       0.0    0.0      0.0        0.0   
50             0.0        0.0    0.0       0.0    0.0      0.0        0.0   
60             0.0        0.0    0.0  911454.0    0.0      0.0        0.0   
64             0.0        0.0    0.0       0.0    0.0      0.0        0.0   
67             0.0        0.0    0.0       0.0    0.0  10427.0        0.0   
68             0.0        0.0    0.0       0.0    0.0      0.0  1186851.0   
79             0.0        0.0    0.0       0.0    0.0      0.0        0.0   
102            0.0        0.0    0.0       0.0    0.0      0.0        0.0   
103            0.0        0.0    0.0       0.0    0.0      0.0        0.0   
104            0.0        0.0    0.0       0.0    0.0      0.0        0.0   